In [ ]:
import json
from multiprocessing import Pool
import platform

import fair
import matplotlib.pyplot as pl
import pandas as pd
import numpy as np
import scipy.stats as st
from tqdm import tqdm
from climateforcing.utils import mkdir_p

## Set up plotting styles

In [ ]:
pl.rcParams['figure.figsize'] = (12/2.54, 9/2.54)
pl.rcParams['font.size'] = 12
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['ytick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.major.left'] = True
pl.rcParams['ytick.major.right'] = True
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['xtick.top'] = True
pl.rcParams['ytick.right'] = True

## RCMIP SSP1-2.6 emissions

In [ ]:
ssp_df = pd.read_csv('../data_input/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
years = np.arange(1750, 2111)
years_future = [2015] + list(range(2020,2111,10))

startyear = 1750
first_scenyear = 2015
last_scenyear = 2110
first_row = int(first_scenyear-startyear)
last_row = int(last_scenyear-startyear)

scenarios = ['ssp126']

species = [  # in fair 1.6, order is important
    '|CO2|MAGICC Fossil and Industrial',
    '|CO2|MAGICC AFOLU',
    '|CH4',
    '|N2O',
    '|Sulfur',
    '|CO',
    '|VOC',
    '|NOx',
    '|BC',
    '|OC',
    '|NH3',
    '|CF4',
    '|C2F6',
    '|C6F14',
    '|HFC23',
    '|HFC32',
    '|HFC4310mee',
    '|HFC125',
    '|HFC134a',
    '|HFC143a',
    '|HFC227ea',
    '|HFC245fa',
    '|SF6',
    '|CFC11',
    '|CFC12',
    '|CFC113',
    '|CFC114',
    '|CFC115',
    '|CCl4',
    '|CH3CCl3',
    '|HCFC22',
    '|HCFC141b',
    '|HCFC142b',
    '|Halon1211',
    '|Halon1202',
    '|Halon1301',
    '|Halon2402',
    '|CH3Br',
    '|CH3Cl',
]

# Assume that units coming out of aneris don't change. One day I'll do unit parsing
unit_convert = np.ones(40)
unit_convert[1] = 12/44/1000
unit_convert[2] = 12/44/1000
unit_convert[4] = 28/44/1000
unit_convert[5] = 32/64
unit_convert[8] = 14/46

data_out = {}

for scenario in scenarios:
    data_out[scenario] = np.ones((361, 40)) * np.nan
    data_out[scenario][:,0] = years

    # past emissions from Zeb
    for i, specie in enumerate(species):
        data_out[scenario][:,i+1] = ssp_df.loc[(ssp_df['Region']=='World')&(ssp_df['Scenario']==scenario)&(ssp_df['Variable'].str.endswith(specie)),str(startyear):'2110'].interpolate(axis=1)*unit_convert[i+1]

In [ ]:
pl.plot(data_out[scenario][250:300, 0], data_out[scenario][250:300, 11])

In [ ]:
results_out = {}

## Load up slim configs

This contains about 3700 ensemble members - we don't use future carbon cycle assessments to constrain the ensemble. Other differences:

- volcanic forcing efficacy of 0.6
- solar forcing zero after 2019

The slim config is a good deal smaller than the AR6 json - but we need a bit of extra code to convert this into what is used in AR6.

In [ ]:
with open('../data_input/ar6-fair-samples/fair-1.6.2-wg3-params-common.json') as f:
    config_list_common = json.load(f)

In [ ]:
with open('../data_input/ar6-fair-samples/fair-1.6.2-wg3-params-slim-reconstrained.json') as f:
    config_list_variable = json.load(f)

In [ ]:
updated_config = []
for i, cfg in enumerate(config_list_variable):
    updated_config.append({})
    for key, value in cfg.items():
        if isinstance(value, list):
            updated_config[i][key] = np.asarray(value)
        else:
            updated_config[i][key] = value
    updated_config[i]['emissions'] = data_out['ssp126']
    updated_config[i]['diagnostics'] = 'AR6'
    updated_config[i]["efficacy"] = np.ones(45)
    updated_config[i]["efficacy"][43] = 0.6
    updated_config[i]["gir_carbon_cycle"] = True
    updated_config[i]["temperature_function"] = "Geoffroy"
    updated_config[i]["aerosol_forcing"] = "aerocom+ghan2"
    updated_config[i]["fixPre1850RCP"] = False
    
    e_pi = np.zeros(40)
    c_pi = np.zeros(31)
    scale = np.ones(45)
    for idx in range(5, 12):
        e_pi[idx] = config_list_common["E_pi"][idx - 5]
    c_pi[0] = cfg["C_pi_CO2"]
    c_pi[1] = config_list_common["C_pi"][0]
    c_pi[2] = config_list_common["C_pi"][1]
    c_pi[3] = config_list_common["C_pi"][2]
    c_pi[20] = config_list_common["C_pi"][3]
    c_pi[25] = config_list_common["C_pi"][4]
    c_pi[29] = config_list_common["C_pi"][5]
    c_pi[30] = config_list_common["C_pi"][6]
    scale[1] = cfg["scale"][0]
    scale[2] = cfg["scale"][1]
    for idx in range(3, 31):
        scale[idx] = cfg["scale"][2]
    scale[15] = scale[15] * config_list_common["cfc11_adj"]
    scale[16] = scale[16] * config_list_common["cfc12_adj"]
    scale[33] = cfg["scale"][3]
    scale[34] = cfg["scale"][4]
    scale[41] = cfg["scale"][5]
    scale[42] = cfg["scale"][6]
    scale[43] = cfg["scale"][7]
    f_solar = np.zeros(361)
    f_solar[:270] = (
        np.linspace(0, cfg["trend_solar"], 270)
        + np.array(config_list_common["default_solar"])[:270] * cfg["scale"][8]
    )
    f_solar[270:] = (
        cfg["trend_solar"]
#        + np.array(config_list_common["default_solar"])[270:351] * cfg["scale"][8]
    )
#    f_solar[351:361] = np.array(config_list_common["default_solar"])[351:]
    
    updated_config[i]["b_aero"] = [
        cfg["b_aero"][0],
        0,
        0,
        0,
        cfg["b_aero"][1],
        cfg["b_aero"][2],
        cfg["b_aero"][3],
    ]
    updated_config[i]["E_pi"] = e_pi
    updated_config[i]["scale"] = scale
    updated_config[i]["F_solar"] = f_solar
    updated_config[i]["F_volcanic"] = np.array(config_list_common["default_volcanic"])
    updated_config[i]["C_pi"] = c_pi
    updated_config[i]["ghg_forcing"] = config_list_common["ghg_forcing"]
    updated_config[i]["aCO2land"] = config_list_common["aCO2land"]
    updated_config[i]["stwv_from_ch4"] = config_list_common["stwv_from_ch4"]
    updated_config[i]["F_ref_BC"] = config_list_common["F_ref_BC"]
    updated_config[i]["E_ref_BC"] = config_list_common["E_ref_BC"]
    updated_config[i]["tropO3_forcing"] = config_list_common["tropO3_forcing"]
    updated_config[i]["natural"] = np.array(config_list_common["natural"])
    
    # delete params that are not arguments to fair
    del updated_config[i]["trend_solar"]
    del updated_config[i]["C_pi_CO2"]

In [ ]:
updated_config[0]

## Run FaIR

In [ ]:
def run_fair(args):
    thisC, thisF, thisT, _, thisOHU, _, thisAF = fair.forward.fair_scm(**args)
    return (thisT, thisF[:,35:41].sum(axis=1))

def fair_process(emissions):
    updated_config = []
    for i, cfg in enumerate(config_list_variable):
        updated_config.append({})
        for key, value in cfg.items():
            if isinstance(value, list):
                updated_config[i][key] = np.asarray(value)
            else:
                updated_config[i][key] = value
        updated_config[i]['emissions'] = emissions
        updated_config[i]['diagnostics'] = 'AR6'
        updated_config[i]["efficacy"] = np.ones(45)
        updated_config[i]["efficacy"][43] = 0.6
        updated_config[i]["gir_carbon_cycle"] = True
        updated_config[i]["temperature_function"] = "Geoffroy"
        updated_config[i]["aerosol_forcing"] = "aerocom+ghan2"
        updated_config[i]["fixPre1850RCP"] = False

        e_pi = np.zeros(40)
        c_pi = np.zeros(31)
        scale = np.ones(45)
        for idx in range(5, 12):
            e_pi[idx] = config_list_common["E_pi"][idx - 5]
        c_pi[0] = cfg["C_pi_CO2"]
        c_pi[1] = config_list_common["C_pi"][0]
        c_pi[2] = config_list_common["C_pi"][1]
        c_pi[3] = config_list_common["C_pi"][2]
        c_pi[20] = config_list_common["C_pi"][3]
        c_pi[25] = config_list_common["C_pi"][4]
        c_pi[29] = config_list_common["C_pi"][5]
        c_pi[30] = config_list_common["C_pi"][6]
        scale[1] = cfg["scale"][0]
        scale[2] = cfg["scale"][1]
        for idx in range(3, 31):
            scale[idx] = cfg["scale"][2]
        scale[15] = scale[15] * config_list_common["cfc11_adj"]
        scale[16] = scale[16] * config_list_common["cfc12_adj"]
        scale[33] = cfg["scale"][3]
        scale[34] = cfg["scale"][4]
        scale[41] = cfg["scale"][5]
        scale[42] = cfg["scale"][6]
        scale[43] = cfg["scale"][7]
        f_solar = np.zeros(361)
        f_solar[:270] = (
            np.linspace(0, cfg["trend_solar"], 270)
            + np.array(config_list_common["default_solar"])[:270] * cfg["scale"][8]
        )
        f_solar[270:] = (
            cfg["trend_solar"]
    #        + np.array(config_list_common["default_solar"])[270:351] * cfg["scale"][8]
        )
    #    f_solar[351:361] = np.array(config_list_common["default_solar"])[351:]

        updated_config[i]["b_aero"] = [
            cfg["b_aero"][0],
            0,
            0,
            0,
            cfg["b_aero"][1],
            cfg["b_aero"][2],
            cfg["b_aero"][3],
        ]
        updated_config[i]["E_pi"] = e_pi
        updated_config[i]["scale"] = scale
        updated_config[i]["F_solar"] = f_solar
        updated_config[i]["F_volcanic"] = np.array(config_list_common["default_volcanic"])
        updated_config[i]["C_pi"] = c_pi
        updated_config[i]["ghg_forcing"] = config_list_common["ghg_forcing"]
        updated_config[i]["aCO2land"] = config_list_common["aCO2land"]
        updated_config[i]["stwv_from_ch4"] = config_list_common["stwv_from_ch4"]
        updated_config[i]["F_ref_BC"] = config_list_common["F_ref_BC"]
        updated_config[i]["E_ref_BC"] = config_list_common["E_ref_BC"]
        updated_config[i]["tropO3_forcing"] = config_list_common["tropO3_forcing"]
        updated_config[i]["natural"] = np.array(config_list_common["natural"])

        # delete params that are not arguments to fair
        del updated_config[i]["trend_solar"]
        del updated_config[i]["C_pi_CO2"]
        
    # multiprocessing is not working for me on Windows
    if platform.system() == 'Windows':
        shape = (361, len(updated_config))
        t = np.ones(shape) * np.nan
        f_aer = np.ones(shape) * np.nan
        for i, cfg in tqdm(enumerate(updated_config), total=len(updated_config), position=0, leave=True):
            t[:,i], f_aer[:,i] = run_fair(updated_config[i])

    else:
        if __name__ == '__main__':
            with Pool(WORKERS) as pool:
                result = list(tqdm(pool.imap(run_fair, updated_config), total=len(updated_config), position=0, leave=True))

        result_t = np.array(result).transpose(1,2,0)
        t, f_aer = result_t
    temp_rebase = t - t[100:151,:].mean(axis=0)

    return temp_rebase, f_aer



results_out = {}
WORKERS = 7  # set this based on your individual machine - allows parallelisation. nprocessors-1 is a sensible shout.

for scenario in tqdm(['ssp126'], position=0, leave=True):
    results_out[scenario] = {}
    (
        results_out[scenario]['temperature'],
        results_out[scenario]['ERFaer']
    ) = fair_process(data_out[scenario])

In [ ]:
for scenario in scenarios:
    for var in ['temperature', 'ERFaer']:
        mkdir_p('../data_output/fair_{}/'.format(var))
        df_out = pd.DataFrame(results_out[scenario][var][245:351,:])
        df_out['year'] = np.arange(1995.5, 2101)
        df_out.set_index('year', inplace=True)
        df_out.to_csv('../data_output/fair_{}/{}.csv'.format(var, scenario), float_format="%6.4f")

In [ ]:
hadcrut5_df = pd.read_csv('../data_input/gsat/HadCRUT.5.0.1.0.analysis.summary_series.global.annual.csv')
hadcrut5_df

In [ ]:
mkdir_p('../plots/')

In [ ]:
n_ens = len(updated_config)

In [ ]:
fig,ax=pl.subplots()
ax.plot(np.arange(1750,2111), np.median(results_out['ssp126']['temperature'], axis=1), color='k', label='FaIR ensemble median')
ax.fill_between(np.arange(1750,2111), np.percentile(results_out['ssp126']['temperature'], 5, axis=1), np.percentile(results_out['ssp126']['temperature'], 95, axis=1), label='FaIR 5-95% range', color="0.7")
ax.plot(hadcrut5_df.loc[0:171,'Time'], hadcrut5_df.loc[0:171,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), color='r', label='HadCRUT5')
ax.set_xlim(1980,2100)
ax.set_ylim(0,2.5)
ax.axhline(1.5, ls=':', color='k')
ax.set_title('SSP1-2.6')
ax.legend(loc='lower right')
ax.set_ylabel('Temperature relative to 1850-1900, °C')
pl.figtext(1,0.02,'@chrisroadmap',ha='right', color='0.6')
pl.figtext(0,0.02,f'{n_ens} ensemble members, FaIRv{fair.__version__}')
#ax.text(0.99,0.01,'FaIRv1.6.2 2237 constrained ensemble members', transform=ax.transAxes, ha='right')
#ax.text(0.99,)
fig.tight_layout(rect=[0.0,0.05,1,1])
fig.patch.set_facecolor('white')
pl.savefig('../plots/ssp126-default.png')

In [ ]:
fig,ax=pl.subplots()
ax.plot(np.arange(1750,2111), np.median(results_out['ssp126']['ERFaer'], axis=1), color='k', label='FaIR ensemble median')
ax.fill_between(np.arange(1750,2111), np.percentile(results_out['ssp126']['ERFaer'], 5, axis=1), np.percentile(results_out['ssp126']['ERFaer'], 95, axis=1), label='FaIR 5-95% range', color="0.7")
#ax.plot(hadcrut5_df.loc[0:170,'Time'], hadcrut5_df.loc[0:170,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), color='r', label='HadCRUT5')
#ax.set_xlim(1980,2100)

In [ ]:
results_out['ssp126']['ERFaer'][269,:].mean()

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=79)
constraint1 = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(results_out['ssp126']['ERFaer'][269,:][i], loc=-1.58, scale=0.10)/st.norm.pdf(-1.58, loc=-1.58, scale=0.10)
    if likelihood>=accept_prob[i]:
        constraint1[i] = True
    #print(likelihood)
np.sum(constraint1)

In [ ]:
pl.hist(results_out['ssp126']['ERFaer'][269,constraint1])

In [ ]:
results_out['ssp126']['ERFaer'][269,constraint1].mean()

In [ ]:
results_out['ssp126']['ERFaer'][269,constraint1].std()

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=80)
constraint2 = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(results_out['ssp126']['ERFaer'][269,:][i], loc=-0.44, scale=0.105)/st.norm.pdf(-0.44, loc=-0.44, scale=0.105)
    if likelihood>=accept_prob[i]:
        constraint2[i] = True
    #print(likelihood)
np.sum(constraint2)

In [ ]:
pl.hist(results_out['ssp126']['ERFaer'][269,constraint2])

In [ ]:
results_out['ssp126']['ERFaer'][269,constraint2].std()

In [ ]:
results_out['ssp126']['ERFaer'][269,constraint2].mean()

In [ ]:
fig,ax=pl.subplots(figsize=(10,10))
ax.plot(np.arange(1750,2111), np.median(results_out['ssp126']['temperature'][:,constraint1], axis=1), color='b')
ax.fill_between(np.arange(1750,2111), np.percentile(results_out['ssp126']['temperature'][:,constraint1], 5, axis=1), np.percentile(results_out['ssp126']['temperature'][:,constraint1], 95, axis=1), color="b", alpha=0.2, label='strong PD Faer (%.2f $\pm$ %.2f) W m$^{-2}$' % (results_out['ssp126']['ERFaer'][269,constraint1].mean(), results_out['ssp126']['ERFaer'][269,constraint1].std()))
ax.plot(np.arange(1750,2111), np.median(results_out['ssp126']['temperature'][:,constraint2], axis=1), color='g')
ax.fill_between(np.arange(1750,2111), np.percentile(results_out['ssp126']['temperature'][:,constraint2], 5, axis=1), np.percentile(results_out['ssp126']['temperature'][:,constraint2], 95, axis=1), color="g", alpha=0.2, label='weak PD Faer (%.2f $\pm$ %.2f) W m$^{-2}$' % (results_out['ssp126']['ERFaer'][269,constraint2].mean(), results_out['ssp126']['ERFaer'][269,constraint1].std()))
ax.plot(np.arange(1750,2111), np.median(results_out['ssp126']['temperature'], axis=1), color='k')
ax.fill_between(np.arange(1750,2111), np.percentile(results_out['ssp126']['temperature'], 5, axis=1), np.percentile(results_out['ssp126']['temperature'], 95, axis=1), label='AR6 ensemble (%.2f $\pm$ %.2f) W m$^{-2}$' % (results_out['ssp126']['ERFaer'][269,:].mean(), results_out['ssp126']['ERFaer'][269,:].std()), color="k", alpha=0.2)
ax.plot(hadcrut5_df.loc[0:171,'Time'], hadcrut5_df.loc[0:171,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), color='r', label='HadCRUT5')
ax.set_xlim(1980,2100)
ax.set_ylim(0,3)
ax.axhline(1.5, ls=':', color='k')
ax.legend(loc='upper left')
ax.set_title('SSP1-2.6')
pl.savefig('../plots/ssp126_three_aer_forcings.png')